In [2]:
import re
from flask import Flask, request, jsonify
from flask_cors import CORS
import psycopg2
import logging
from psycopg2 import sql
from word2number import w2n

app = Flask(__name__)
CORS(app)  # This will enable CORS for all routes

def get_db_connection():
    connection = psycopg2.connect(
        dbname="gisprddb",
        user="prdgis_user",
        password="Gis@app#999",
        host="20.0.4.237",
        port="5432"
    )
    return connection

# Example function to process AI predictions based on voice input
def predict_action(command):
    # Define the action map with regex patterns for dynamic extraction
    action_map = {
        "zoomOut": ["zoom out", "reduce zoom"],
        "clear": ["clear", "clearall", "clear all"],
        "wardboundary": [r"ward\s(\d+)", r"ward\sboundary\s(\d+)" , r"boundary\s(\d+)" , r"what\sboundary\s(\d+)"],
        "number": [r"number\s(\d+)"],
        "flyToNYC": ["code", "fly to New York"],
        "flyToLA": ["fly to Los Angeles", "visit LA"],
        "threedpsPowerPlant": ["power plant", "electric station"],
        "threedpsTransformers": ["how many transformers", "step down transformers", "step down transformer", "stp transformers", "transformer"],
        "linesConnect": ["connect", "connected"],
        "threedpsHTPoles": ["ht poles", "hd poles"],
        "threedpsSubstation": ["substation", "sub station"],
        "threedpsLTPoles": ["lt poles","poles"],
        "threedpsSwitch": ["switch"],
        "threedpsHouse": ["house", "home", "house house"]
    }
    
    # Define the corresponding action details
    actions = {
        "zoomOut": {"action": "zoomOut"},
        "clear": {"action": "clear"},
        "flyToNYC": {"action": "flyTo", "parameters": {"location": {"longitude": -74.005974, "latitude": 40.712776}}},
        "flyToLA": {"action": "flyTo", "parameters": {"location": {"longitude": -118.243683, "latitude": 34.052235}}},
        "threedpsPowerPlant": {"action": "threedps", "parameters": {"location": {"layername": 'powerplant', "image": 'pplant.png'}}},
        "threedpsTransformers": {"action": "threedps", "parameters": {"location": {"layername": 'transformer', "image": 'SDtrans.png'}}},
        "linesConnect": {"action": "lines", "parameters": {"location": {"layername": 'threedstpline', "color": 'Cesium.Color.RED'}}},
        "threedpsHTPoles": {"action": "threedps", "parameters": {"location": {"layername": 'threedthpole', "image": 'power.png'}}},
        "threedpsSubstation": {"action": "threedps", "parameters": {"location": {"layername": 'substation', "image": 'pt.png'}}},
        "threedpsLTPoles": {"action": "threedps", "parameters": {"location": {"layername": 'poles', "image": 'ec.png'}}},
        "threedpsSwitch": {"action": "threedps", "parameters": {"location": {"layername": 'switch', "image": 'electricpole.png'}}},
        "threedpsHouse": {"action": "threedps", "parameters": {"location": {"layername": 'house', "image": 'house.png'}}}
    }

    for action, patterns in action_map.items():
        for pattern in patterns:
            match = re.search(pattern, command)
            if match:
                if action == "wardboundary":
                    ward_number = match.group(1)
                    return {"action": "wardboundary", "parameters": {"ward_number": ward_number}}
                elif action == "number":
                    housid = match.group(1)
                    return {"action": "number", "parameters": {"housid": housid}}
                return actions[action]
    
    return {"action": "unknown"}

@app.route('/allpredictsone', methods=['POST'])
def predict():
    data = request.json
    command_input = data.get("command")

    action_response = predict_action(command_input)

    if action_response["action"] == "wardboundary":
        ward_number = action_response["parameters"]["ward_number"]
        ward_url = f"https://gisportal.smartkochi.in/cgi-bin/mapserv?map=/var/www/codegeo/Ernakulam/3DBuildings.map&SERVICE=WFS&VERSION=1.0.0&REQUEST=getfeature&TYPENAME=COCHIWARDQUERY2&outputformat=geojson&country={ward_number}"
        action_response["url"] = ward_url
        return jsonify(action_response)
    elif action_response["action"] == "number":
        housid = action_response["parameters"]["housid"]
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            query = sql.SQL("""
                SELECT housid, mh.poleid, ms.bdtransid
                FROM geodbkochi.ml_switch ms
                JOIN geodbkochi.house mh ON mh.poleid::int = ms.id::int
                JOIN geodbkochi.ml_sdtransformer sdt ON sdt.tid = ms.bdtransid
                WHERE housid = %s
            """)
            cursor.execute(query, [housid])
            result = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            result_json = [dict(zip(columns, row)) for row in result]

            if result_json:
                result_item = result_json[0]
                bdtransid = result_item.get("bdtransid")
                poleid = result_item.get("poleid")

                layername = action_response.get("parameters", {}).get("location", {}).get("layername", "")
                url = f"https://gisportal.smartkochi.in/cgi-bin/mapserv?map=/var/www/codegeo/threedwfstest.map&SERVICE=WFS&VERSION=1.0.0&REQUEST=getfeature&TYPENAME={layername}&outputformat=geojson&id={housid}"
                bdtransid_url = f"https://gisportal.smartkochi.in/cgi-bin/mapserv?map=/var/www/codegeo/threedwfstest.map&SERVICE=WFS&VERSION=1.0.0&REQUEST=getfeature&TYPENAME=sdtransformer&outputformat=geojson&id={bdtransid}"
                poleid_url = f"https://gisportal.smartkochi.in/cgi-bin/mapserv?map=/var/www/codegeo/threedwfstest.map&SERVICE=WFS&VERSION=1.0.0&REQUEST=getfeature&TYPENAME=bdopoles&outputformat=geojson&id={poleid}"

                response = {
                    "action": action_response,
                    "url": url,
                    "bdtransid_url": bdtransid_url,
                    "poleid_url": poleid_url,
                    "query_result": result_json
                }
            else:
                response = {
                    "action": action_response,
                    "url": "",
                    "bdtransid_url": "",
                    "poleid_url": "",
                    "query_result": result_json
                }
            return jsonify(response)
        except psycopg2.Error as e:
            print(f"Database error: {e}")
            return jsonify({"error": str(e)}), 500
        finally:
            if cursor:
                cursor.close()
            if connection:
                connection.close()
    else:
        return jsonify(action_response)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8083)


127.0.0.1 - - [14/Jul/2024 22:16:34] "OPTIONS /allpredictsone HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2024 22:16:34] "POST /allpredictsone HTTP/1.1" 200 -
